In [ ]:
from nustar_pysolar import planning, io
import astropy.units as u
import warnings
warnings.filterwarnings('ignore')

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [ ]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [ ]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [ ]:
tstart = '2018-05-29T15:37:00'
tend = '2018-05-29T23:10:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [ ]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [ ]:
offset = [0., 0.]*u.arcsec

# Loop over each orbit and correct the pointing for the same heliocentric pointing position.

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

In [ ]:
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_skyfield_position(midTime, offset, parallax_correction=True)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")

# Below is prototypes for making mosaics.

## There are current funky skyfield issues happening, but you can use the make_mosaic command with an orbit of the type loaded above to produce a 5x5 mosaic tile.



In [ ]:
def make_mosaic(orbit, outfile='mosaic.txt', write_output=False, make_regions=False):
    ''' 
    Code to make a mosaic for a 4x4 tiled array on the Sun.
    
    Input:
    
    tstart = '2018-05-28T15:37:00'
    tend = '2018-05-28T23:10:00'
    
    positions = make_mosaic(tstart, tend, write_output=True)
    
    Optional flags:
    
    write_output = [False] / True
        Write the output pointing positions in NuSTAR SOC readable formats in 'outfile' for all of the pointings.

    outfile = ['mosaic.txt']
        Output file if write_output is used.
    
    make_regions: [False] / True
        Make ds9 region files for each tile so that you can see how the FoV moves with each mosaic location.


    '''
    import numpy as np
    box_pa = planning.get_nustar_roll(orbit[0], 0)
    pa = box_pa + 90*u.deg
    
    print('Step PA', pa)

    base = np.array([-1.45, -0.725, 0, 0.725, 1.45])
    xsteps = np.append(base, np.flip(base, 0))
    xsteps = np.append(xsteps, base)
    xsteps = np.append(xsteps, np.flip(base, 0))
    xsteps = np.append(xsteps, base)

    ysteps = np.array(np.zeros(5) + 1.45)
    ysteps = np.append(ysteps, np.zeros(5) + 0.725)
    ysteps = np.append(ysteps, np.zeros(5))
    ysteps = np.append(ysteps, np.zeros(5)-0.725)
    ysteps = np.append(ysteps, np.zeros(5)-1.45)

    
    dt = (orbit[1] - orbit[0]) / 25.

    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print("Dwell per position:", dt)
    print("")
    print("NuSTAR Roll Angle to get DET0 in top right {:.02f} deg".format(box_pa.value))
    print("")


    if write_output is True:
        f = open(outfile, 'w')
    
    aim_time = orbit[0]
    for ind, pair in enumerate(zip(xsteps, ysteps)):
        arrive_time = aim_time
        aim_time = aim_time + dt
        # Rotate to the correct PA angle

        delx = -pair[0]*10/60.
        dely = pair[1]*10/60
        
        offset = [0., 0.]*u.deg
        sun_pos = planning.get_skyfield_position(aim_time, offset, load_path='../data', parallax_correction=True)
#        print('Sun time: {} RA (deg): {} Dec (deg): {}'.format(aim_time.isoformat(), sun_pos[0], sun_pos[1]))
        
        offset = [-delx,dely]*u.deg
        sky_pos = planning.get_skyfield_position(aim_time, offset, load_path='../data', parallax_correction=True)
#        print('Arrive time: {} RA (deg): {} Dec (deg): {}'.format(arrive_time.isoformat(), sky_pos[0], sky_pos[1]))

        if make_regions:
            make_test_region(sky_pos[0], sky_pos[1], box_pa, sun_pos[0], sun_pos[1],box_pa, outname='testbox{}.reg'.format(ind))
        if write_output:            
            f.write('{0} {1:.4f} {2:.4f} {3:.4f} {4:.4f}\n'.format(arrive_time.strftime('%Y:%j:%H:%M:%S'),
                                                   sky_pos[0].value, sky_pos[1].value,
                                                    sun_pos[0].value, sun_pos[1].value))


        
    if write_output:
        f.close()


In [ ]:
def make_test_region(boxra, boxdec, boxpa, sunra, sundec, sunpa, outname='testbox.reg'):
    
    f = open(outname, 'w')
    f.write("# Region file format: DS9 version 4.1\n")
    f.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 \
source=1 \n')
    f.write('fk5 \n')
    
    outstring = 'box({0}, {1}, 720", 720", {2})'.format(boxra.value, boxdec.value, boxpa.value+90 % 360)
    f.write(outstring+'\n')
    
    outstring = 'circle({0}, {1}, 960.5")'.format(sunra.value, sundec.value)


    f.write(outstring+'\n')

    outstring = 'vector({0}, {1}, 960.5", {2})'.format(sunra.value, sundec.value, sunpa.value+90 % 360 )
    f.write(outstring+'\n')

    f.close
    return



# This is where you actually make the Mosaic

In [ ]:
# Just use the first orbit...or choose one. This may download a ton of deltat.preds, which is a known 
# bug to be fixed.

orbit = orbits[0]
make_mosaic(orbits[0], write_output=True)